# Simple Chatbot Gradio + Google Gemini API 🚀

In [17]:
#@title Mostrar Imagen
from IPython.display import Image
url = 'https://github.com/AleNunezArroyo/Medium/blob/main/img/BannerM4.png?raw=true'
Image(url=url, width=800)

## Hello world with Gemini API

Install the Python SDK

In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.6/663.6 kB 24.3 MB/s eta 0:00:00


The library and API Key information is added

In [2]:
import google.generativeai as genai
GOOGLE_API_KEY = "your-api"
genai.configure(api_key=GOOGLE_API_KEY)

Model configuration

In [3]:
model = genai.GenerativeModel('gemini-pro')

Generate text from text inputs

In [4]:
%%time
question = 'What is the capital of Argentina?'
response = model.generate_content(question)
response.resolve()
response.text

CPU times: user 47.4 ms, sys: 7.19 ms, total: 54.6 ms
Wall time: 3.25 s


'Buenos Aires'

## Chat conversations

Gemini enables to have a [conversation](https://ai.google.dev/tutorials/python_quickstart#chat_conversations). The `ChatSession` class simplifies the process by managing the state of the conversation, so unlike with `generate_content`, you do not have to store the conversation history as a list.

In [5]:
chat = model.start_chat(history=[])

In [6]:
countries = ['Chile', 'Bolivia']

for country in countries:
    message = f'What is the capital of {country}?'
    chat.send_message(message)

In [7]:
chat.history

[parts {
   text: "What is the capital of Chile?"
 }
 role: "user",
 parts {
   text: "Santiago"
 }
 role: "model",
 parts {
   text: "What is the capital of Bolivia?"
 }
 role: "user",
 parts {
   text: "Sucre (constitutional capital)\nLa Paz (administrative capital)"
 }
 role: "model"]

In [8]:
response = chat.send_message('From which countries did I ask you the capital city?')
response.resolve()
response.text

'Chile and Bolivia\n\nIs there anything else I can help you with today?'

In [9]:
chat.history

[parts {
   text: "What is the capital of Chile?"
 }
 role: "user",
 parts {
   text: "Santiago"
 }
 role: "model",
 parts {
   text: "What is the capital of Bolivia?"
 }
 role: "user",
 parts {
   text: "Sucre (constitutional capital)\nLa Paz (administrative capital)"
 }
 role: "model",
 parts {
   text: "From which countries did I ask you the capital city?"
 }
 role: "user",
 parts {
   text: "Chile and Bolivia\n\nIs there anything else I can help you with today?"
 }
 role: "model"]

## Streaming Gradio Chatbots

Documentation: https://www.gradio.app/guides/creating-a-chatbot-fast

In [10]:
!pip install --upgrade gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.9 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl s

In [14]:
import time
import gradio as gr

def slow_echo(message, history):
    for i in range(len(message)):
        time.sleep(0.3)
        yield "You typed: " + message[: i+1]

gr.ChatInterface(slow_echo).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c3769840823fc9ef84.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [12]:
import time
import gradio as gr

chat = model.start_chat(history=[])

# Transform Gradio history to Gemini format
def transform_history(history):
    new_history = []
    for chat in history:
        new_history.append({"parts": [{"text": chat[0]}], "role": "user"})
        new_history.append({"parts": [{"text": chat[1]}], "role": "model"})
    return new_history

def response(message, history):
    global chat
    # The history will be the same as in Gradio, the 'Undo' and 'Clear' buttons will work correctly.
    chat.history = transform_history(history)
    response = chat.send_message(message)
    response.resolve()

    # Each character of the answer is displayed
    for i in range(len(response.text)):
        time.sleep(0.05)
        yield response.text[: i+1]

gr.ChatInterface(response,
                 title='Gemini Chat',
                 textbox=gr.Textbox(placeholder="Question to Gemini"),
                 retry_btn=None).launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8fdfb12203db75a030.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://910e7c6794b413cd3d.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://8fdfb12203db75a030.gradio.live


In [13]:
chat.history

[parts {
   text: "What is the capital of Bolivia?"
 }
 role: "user",
 parts {
   text: "Sucre"
 }
 role: "model",
 parts {
   text: "How many people does the city have? "
 }
 role: "user",
 parts {
   text: "295,323 (2020)"
 }
 role: "model"]